In [1]:
import os
import numpy as np
import pandas as pd
from twarc import Twarc2
import re
import copy
import spacy
from spacy.lang.en import stop_words
from pysentimiento.preprocessing import preprocess_tweet

C:\Users\user\.conda\envs\chatgpt\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\user\.conda\envs\chatgpt\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
C:\Users\user\.conda\envs\chatgpt\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
data_root_dir_path = r"C:\Users\user\Datasets\Twitter"

## Get Twitter data from Twitter API using hydration (if original Tweet text is not available)

In [29]:
tweet_id_file = "" # Name of dataset with Tweet IDs (csv format)

In [30]:
tweet_ids_to_hydrate = pd.read_csv(os.path.join(data_root_dir_path,tweet_id_file ), delimiter='\t')

In [31]:
tweet_ids_to_hydrate.head()

,tweet_id,date,time
0,1243388154213588996,2020-3-27,4:3:31
1,1243388154385580034,2020-3-27,4:3:31
2,1243388154490392577,2020-3-27,4:3:31
3,1243388154582716422,2020-3-27,4:3:31
4,1243388154737901571,2020-3-27,4:3:31


In [32]:
tweet_ids_to_hydrate["tweet_id"].to_csv(os.path.join(data_root_dir_path, 'tweet_ids_to_hydrate.txt'), sep=',', index=False)

In [34]:
# Using the command line, hydrate the tweet IDs using the "Twarc" Python Library  (pip install twarc)
# First create a configuration file with all the Twitter API keys (Bearer Token, API key and API secret key) using the command: "twarc2 configure" and follow the written instructions.
# Full documentation of the Twarc Library at: https://twarc-project.readthedocs.io/en/latest/tutorial/

# twarc2 hydrate tweet_ids_to_hydrate.txt > hydrated_tweets.json

## Load Twitter data containing tweets

In [3]:
# Dataset was oibtained from https://www.kaggle.com/datasets/datatattle/covid-19-nlp-text-classification/
tweets_file_name = os.path.join(data_root_dir_path, "Corona_NLP_train.csv")
tweets_df = pd.read_csv(tweets_file_name, engine='python', encoding = "ISO-8859-1")
tweets_df.head()

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,3799,48751,London,16-03-2020,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral
1,3800,48752,UK,16-03-2020,advice Talk to your neighbours family to excha...,Positive
2,3801,48753,Vagabonds,16-03-2020,Coronavirus Australia: Woolworths to give elde...,Positive
3,3802,48754,NaN,16-03-2020,My food stock is not the only one which is emp...,Positive
4,3803,48755,NaN,16-03-2020,"Me, ready to go at supermarket during the #COV...",Extremely Negative


## Quick text pre-processing 

In [4]:
extended_stop_wrods = list(stop_words.STOP_WORDS)

In [5]:
def get_url_patern():
    return re.compile(
        r'(https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9][a-za-z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|https?:\/\/(?:www\.|(?!www))'
        r'[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9]\.[^\s]{2,})')


def get_hashtags_pattern():
    return re.compile(r'#\w+')


def get_single_letter_words_pattern():
    return re.compile(r'(?<![\w-])\w(?![\w-])')


def get_blank_spaces_pattern():
    return re.compile(r'\s{2,}|\t')


def get_twitter_reserved_words_pattern():
    return re.compile(r'(RT|rt|FAV|fav|VIA|via)')


def get_mentions_pattern():
    return re.compile(r'@\w\*')


def clean_sentence_for_sentiment_analysis(sentence, remove_urls=True, remove_retweets=True, remove_stop_words=False, handle_hashtags=True, remove_short_words=False, remove_punctuation=False,
                                          remove_numbers=True, decode_ascii=False, max_text_length=2048):
    if remove_urls:
        sentence = re.sub(pattern=get_url_patern(), repl="", string=sentence)
        sentence = re.sub(r'http\S+', "", sentence)  # remove http links
        sentence = re.sub(r'bit.ly/\S+', "", sentence)  # remove bitly links
        sentence = sentence.strip('[link]')  # remove [links]
    if remove_retweets:
        sentence = re.sub(pattern=get_mentions_pattern(), repl="", string=sentence)
        sentence = re.sub(pattern=get_twitter_reserved_words_pattern(), repl='', string=sentence)
        sentence = re.sub('(RT\s@[A-Za-z]+[A-Za-z0-9-*]+)', "", sentence)  # remove retweet
        sentence = re.sub('(@[A-Za-z]+[A-Za-z0-9-_]+)', "", sentence)  # remove tweeted at
    if remove_numbers:
        sentence = re.sub('[0-9]+', ' ', sentence)
    if remove_stop_words:
        sentence = ' '.join([word for word in sentence.split() if word not in extended_stop_wrods])
    if remove_short_words:
        sentence = ' '.join([word for word in sentence.split() if (not len(word) <= 2)])# or (word in UNICODE_EMOJI)])
    if handle_hashtags:
        sentence = preprocess_tweet(sentence, lang="en", demoji=False)
    if remove_punctuation:
        sentence = sentence.translate(str.maketrans('', '', string.punctuation))  # remove punctuation
    if decode_ascii:
        sentence=sentence.encode('ascii', 'ignore').decode('ascii')
        sentence = sentence.replace("\r", "")
    sentence = sentence[0:max_text_length]
    sentence = sentence.strip()

    clean_sentence = sentence
    return clean_sentence

In [6]:
tweets_df["clean_post"] = tweets_df["OriginalTweet"].apply(lambda x: clean_sentence_for_sentiment_analysis(x,
                                                                                                           remove_stop_words=False,
                                                                                                           remove_urls=True,
                                                                                                           handle_hashtags=True,
                                                                                                           remove_retweets=True,
                                                                                                           remove_short_words=False,
                                                                                                           remove_punctuation=False,
                                                                                                           remove_numbers=False,
                                                                                                           decode_ascii=True))
print("Number of tweets BEFORE pre-processing", len(tweets_df["clean_post"]))

Number of tweets BEFORE pre-processing 41157


In [7]:
tweets_df = tweets_df[tweets_df["clean_post"].str.split().str.len() > 2]
tweets_df = tweets_df[tweets_df["clean_post"].str.count('\?') < 5]
print("Number of tweets AFTER pre-processing", len(tweets_df["clean_post"]))
tweets_df.head()

Number of tweets AFTER pre-processing 40596


,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment,clean_post
1,3800,48752,UK,16-03-2020,advice Talk to your neighbours family to excha...,Positive,advice Talk to your neighbours family to excha...
2,3801,48753,Vagabonds,16-03-2020,Coronavirus Australia: Woolworths to give elde...,Positive,Coronavirus Australia: Woolwohs to give elderl...
3,3802,48754,NaN,16-03-2020,My food stock is not the only one which is emp...,Positive,My food stock is not the only one which is emp...
4,3803,48755,NaN,16-03-2020,"Me, ready to go at supermarket during the #COV...",Extremely Negative,"Me, ready to go at supermarket during the hash..."
5,3804,48756,"ÃT: 36.319708,-82.363649",16-03-2020,As news of the regionÂs first confirmed COVID...,Positive,As news of the regions first confirmed COVID-1...


In [8]:
tweets_df["Sentiment"] = tweets_df["Sentiment"].replace("Extremely Negative", "Negative")
tweets_df["Sentiment"] = tweets_df["Sentiment"].replace("Extremely Positive", "Positive")

In [9]:
tweets_df.head()

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment,clean_post
1,3800,48752,UK,16-03-2020,advice Talk to your neighbours family to excha...,Positive,advice Talk to your neighbours family to excha...
2,3801,48753,Vagabonds,16-03-2020,Coronavirus Australia: Woolworths to give elde...,Positive,Coronavirus Australia: Woolwohs to give elderl...
3,3802,48754,NaN,16-03-2020,My food stock is not the only one which is emp...,Positive,My food stock is not the only one which is emp...
4,3803,48755,NaN,16-03-2020,"Me, ready to go at supermarket during the #COV...",Negative,"Me, ready to go at supermarket during the hash..."
5,3804,48756,"ÃT: 36.319708,-82.363649",16-03-2020,As news of the regionÂs first confirmed COVID...,Positive,As news of the regions first confirmed COVID-1...


## Save processed dataset

In [11]:
tweets_df.to_csv(os.path.join(data_root_dir_path, "./Corona_NLP_train_clean.csv"), index=False)
print("Finished!")

Finished!
